Copyright 2020-2022 Universidad Complutense de Madrid

This file software has been employed to calibrate optical spectroscopic data from the OSIRIS instrument at GTC (see [Paliya et al. 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...903L...8P/abstract))

Authors: Cristina Cabello (criscabe@ucm.es), Nicolás Cardiel (cardiel@ucm.es) Jesús Gallego (j.gallego@ucm.es)

SPDX-License-Identifier: GPL-3.0+ License-Filename: LICENSE.txt

In [1]:
import matplotlib.pyplot as plt

from astropy.io import fits
import numpy as np

# <span style="color:red"> Reduction of the OSIRIS standard star spectrum</span>  

# 1) Spectrophotometric standard star: image registration

<span style="color:darkblue"> We had a look at the standard star images:</span>


```
$ fitsheader -k object -k exptime -f

                         filename                             OBJECT   EXPTIME
---------------------------------------------------------- ----------- -------
0002581297-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits SPSTD_GD153     0.5
0002581298-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits SPSTD_GD153     0.5
0002581299-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits SPSTD_GD153    80.0
```

<span style="color:darkblue"> We have 1 exposure of 80 seconds. This image was bias subtracted, trimmed, rotated and divided by the MasterFlat. </span>


In [ ]:
# read median bias frame

with fits.open('../bias/bias_median.fits') as hdul:
    header = hdul[0].header
    bias = hdul[0].data



# read trimmed and rotated normalized flat

with fits.open('../flat/flat_norm.fits') as hdul:
    header = hdul[0].header
    flat_norm = hdul[0].data

    
for i in [99]:
    fname = '00025812{}-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits'.format(i)
    with fits.open(fname) as hdul:
        header = hdul[2].header
        data = hdul[2].data
        
    # subtract bias
    data = data - bias
    
    # trim image
    data = data[:, 25:950]
    
    # rotate image
    data = np.rot90(data, 1)
    
    # divide by flatfield
    data = data / flat_norm

    # save image
    hdu = fits.PrimaryHDU(data, header=header)
    hdulist = fits.HDUList(hdu)
    outfile = '{}brf.fits'.format(i)
    hdulist.writeto(outfile, overwrite=True) 

<span style="color:darkblue"> Furthermore, we used the cleanest (https://cleanest.readthedocs.io/en/latest/) software [(Cardiel 2020)](https://ui.adsabs.harvard.edu/abs/2020ASPC..522..723C/abstract) to interpolate the defective pixels in the individual image.</span>

```
$ cleanest: 99brf.fits -> 99brfc.fits
```

# 2) REDUCEME

<span style="color:darkblue"> We used the ReDucEme (https://reduceme.readthedocs.io/en/latest/) reduction package [(Cardiel 1999)](https://ui.adsabs.harvard.edu/abs/1999PhDT........12C/abstract) for the further reduction steps.</span>


<span style="color:darkblue"> First, we converted the resulting image from FITS format to REDUCEME format. </span>

```
$ R5-leefits_new
*******************************************************************************
REDUCEMEv5.0                Welcome to leefits_new       Version: 01-April-2004
-------------------------------------------------------------------------------


Input FITS file name? 99brfc.fits

-> number of HDUs:            1

> CRPIX1:   -598.500000    
> CRVAL1:    194.305084    

> BITPIX:          -64
> NAXIS:            2
> NAXIS1:         2051
> NAXIS2:          925

Please wait (reading FITS file)...  ..OK!

> STWV:    0.00000000    
> DISP:    0.00000000    

Save whole frame (y/n) [y] ? 
Output file name? 99brfc.u
>>> DATAMIN:   -61.0875015    
>>> DATAMAX:    18645.4570    
Please wait (saving file)...  ..OK!
```


### - 2.1) C-distortion correction and wavelength calibration

<span style="color:darkblue"> We applied the C-distortion correction and the wavelength calibration previously computed (see `00_calibrations.ipynb`) using the ‘rebincw’ function:</span>

```
$ R5-rebincw 
*******************************************************************************
REDUCEMEv5.0                  Welcome to rebincw       Version: 8-December-1996
-------------------------------------------------------------------------------


-------------------------------------------------------------------------------
                               Program REBINCW
    This program needs data (files) generated through FITCDIS and FITLIN.
    Since these files are suposed to be in a directory containing all the
    files from arc reduction, REBINCW will search for the output files of
    FITCIDS and FITLIN  in the current directory first, and,  if they are
    not present there, the search will continue in the directory ../cuar/.
-------------------------------------------------------------------------------

Work with error images (y/n) [n] ? 
Input file name? 99brfc.u
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 0.00000000
>>> DISP  : 0.00000000
>>> OBJECT: [not found]

Correct from C-distortion (y/n) [y] ? 
Number of arcs to be used  (1,...,2) [1] ? 

ARC #     1

>>> C-DISTORTION CORRECTION:
Spectra for which the corr. was derived: from... to... [1,925] ? 
Order of polynomial  (0,...,19)? 2
How many polynomial to be applied (1=no iteration) [1] ? 
Iteration # 1: Polynomial file name (from fitcdis)? ../arc/arc.cdis1

>>> WAVELENGTH CALIBRATION:
Polynomial file name (from fitlin)? ../arc/polarc
>>> Polynomial degree:            7


Enter STWV (central wavelenght of pixel 1)? 3644.58765
Enter desired dispersion (angstroms/pixel)? 2.06242371

Pixel     1    -------> Blank from scan     1 to     1
Pixel     2    -------> Blank from scan     1 to     1
Pixel     3    -------> Blank from scan     1 to     1
Pixel     4    -------> Blank from scan     1 to     1
Pixel     5    -------> Blank from scan     1 to     1
Pixel     6    -------> Blank from scan     1 to     1
Pixel     7    -------> Blank from scan     1 to     1
Pixel     8    -------> Blank from scan     1 to     1
Pixel     9    -------> Blank from scan     1 to     1
Pixel    10    -------> Blank from scan     1 to     1
Pixel    11    -------> Blank from scan     1 to     1
Pixel    12    -------> Blank from scan     1 to     1
Pixel    13    -------> Blank from scan     1 to     1
Pixel    14    -------> Blank from scan     1 to     1
Pixel    15    -------> Blank from scan     1 to     1
Pixel    16    -------> Blank from scan     1 to     1
Pixel    17    -------> Blank from scan     1 to     1
Pixel    18    -------> Blank from scan     1 to     1

>>> Checking conservation of number of counts...   ...OK!
>>> Maximum error (%):    2.09671831    
    in scan no.:    694

Shift spectra (radial velocitiy) (y/n) [n] ? 

Output file name? 99brfc.uw

```


### - 2.2) Sky subtraction

<span style="color:darkblue"> We carried out the sky subtraction by selecting a sky region above and below the star spectrum with the ‘skysubm’ function:</span>

```
$ R5-skysubm
*******************************************************************************
REDUCEMEv5.0                  Welcome to skysubm     Version: 07-September-2007
-------------------------------------------------------------------------------

Work with error images (y/n) [y] ? n
Do you want to estimate errors from the r.m.s. in the sky fits (y/n) [y] ? 
Graphic device #1 (? to see list) [/XServe] ? 
Graphic device #2 (NONE=EXIT) (? to see list) [NONE] ? 
Input file name......? 99brfc.uw
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: [not found]
* Enter first and last channel to compute averaged spatial profile:
Channels [1,2051] ? 

(1) Change x/y-limits
(2) Compute new spatial profile
(0) continue
Option (0/1/2) [0] ? 1
XMIN, XMAX (0,0=automatic) [0,0] ? 
YMIN, YMAX (0,0=automatic) [0,0] ? 0, 1000

(1) Change x/y-limits
(2) Compute new spatial profile
(0) continue
Option (0/1/2) [0] ? 
Select sky regions with [m]ouse or [k]eyboard (m/k) [m] ? k
No. of sky regions to be employed? 2

Region # 1
left limit, right limit? 500,600

Region # 2
left limit, right limit? 800,900

Plot individual fits (y/n) [y] ? 
Polynomial degree  (0,...,19)? 1
Times sigma to remove points [3.0] ? 

ENTER CHANNEL NUMBER TO BE FITTED
 N=Channel number (1.LE.N.LE.NCHAN)
 0=exit
Channel? 1000
-> Dotted line (yellow): error from r.m.s.

ENTER CHANNEL NUMBER TO BE FITTED
 N=Channel number (1.LE.N.LE.NCHAN)
 0=exit
-1=change pol. degree
-2=change sky regions
-3=change limits of previous plot
Channel? 0

Final Sky Region Settings:
Region #01      500   600
Region #02      800   900
Polynomial degree:  1


Polynomial degree  (0,...,19) [1] ? 
Times sigma to remove points [3.00000000] ? 

* Fitting sky...  ...OK
Save sky fit into file (y/n) [n] ? y
Output sky fit file name? 99brfc.sky
Output file name? 99brfc.uws
Output file name for error estimates computed from r.m.s. in fits ? 99brfce.uws
```

### - 2.3) S-distortion correction

<span style="color:darkblue"> We used the ‘sdistor’ function to rectify the shape of the spectrum by correcting for distortion S. </span>
 


```
$ R5-sdistor
*******************************************************************************
REDUCEMEv5.0                  Welcome to sdistor         Version: 10-March-2005
-------------------------------------------------------------------------------

Work with error images (y/n) [y] ? 
Graphic device #1 (? to see list) [/XServe] ? 
Graphic device #2 (NONE=EXIT) (? to see list) [NONE] ? 
Input file name......? 99brfc.uws
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: [not found]
Input error file name [99brfce.uws] ? 
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: [not found] @ERROR@
Perform fit to:
    1 : Simmetric central region
    2 : More general region
    3 : Use user-defined polynomial for distortion
Option (1/2/3) [1] ? 
Channel region to plot averaged spatial profile [1,2051] ? 
>>> Maximum is located at scan:    701
Search for a different maximum (y/n) [n] ? 
Center scan  (1,...,925) [701] ? 
No. of scans at each side to find/fit maximum [3] ? 5
Enter channels to be fitted (0,0=EXIT):
Channel region [0,0] ? 10,2040
Channel region [0,0] ? 
Binning (y/n) [n] ? y
Bin width[1] ? 10
No. of scans around center scan to be plotted  (1,...,925) [4] ? 10
Plot fits to Cauchy functions for each channel (y/n) [n] ? 
>>> Original center scan:          701
>>> Fitted center scan..:          701
Working...
Press RETURN to continue...
Mean deviation around central scan (scans):    1.43928432    
Polynomial degree (<20,-2=RESTART,-3=QUIT)? 2
No. of SCANS from center scan to exclude points [5] ? 
No. of points too far from the center scan:      0
Times SIGMA to exclude points [3.0] ? 
No. of points removed from fit:      0
Coefficients from fit:
> a(0) =    696.241272    
> a(1) =    8.25898629E-03
> a(2) =   -2.71937756E-06
Mean dispersion around polynomial  :   0.113674246    

Change polynomial degree (y/n) [n] ? y
Mean deviation around central scan (scans):    1.43928432    
Polynomial degree (<20,-2=RESTART,-3=QUIT)? 3
No. of SCANS from center scan to exclude points [5] ? 
No. of points too far from the center scan:      0
Times SIGMA to exclude points [3.0] ? 
No. of points removed from fit:      0
Coefficients from fit:
> a(0) =    695.960022    
> a(1) =    9.86172911E-03
> a(2) =   -4.65686844E-06
> a(3) =    6.27440266E-10
Mean dispersion around polynomial  :    5.25294580E-02

Change polynomial degree (y/n) [n] ? 
Working...
Press RETURN to continue...
Mean deviation around central scan (scans):    6.32982478E-02
Polynomial degree (<20,-1=NO MORE FITS,-3=QUIT)? -1
Save input file after initial correction (y/n) [n] ? 

* Plotting fits to local polynomials:
Channel (0=EXIT) (0,...,2051)? 0
No. of iterations to link local fits  (0,...,100) [0] ? 
Working with initial frame...
channell:    100...
channell:    200...
channell:    300...
channell:    400...
channell:    500...
channell:    600...
channell:    700...
channell:    800...
channell:    900...
channell:   1000...
channell:   1100...
channell:   1200...
channell:   1300...
channell:   1400...
channell:   1500...
channell:   1600...
channell:   1700...
channell:   1800...
channell:   1900...
channell:   2000...   ...OK!
Working with error frame...
channell:    100...
channell:    200...
channell:    300...
channell:    400...
channell:    500...
channell:    600...
channell:    700...
channell:    800...
channell:    900...
channell:   1000...
channell:   1100...
channell:   1200...
channell:   1300...
channell:   1400...
channell:   1500...
channell:   1600...
channell:   1700...
channell:   1800...
channell:   1900...
channell:   2000...   ...OK!
Output file name.......(corrected from S-distortion)? 99brfc.uwss
Output error file name (corrected from S-distortion) [99brfce.uwss] ?

```


### - 2.4) 1D spectrum extraction

<span style="color:darkblue"> For the 1D spectrum extraction, we first fitted the star spatial profile with the ‘plots’ package: </span>
 
```
$ R5-plots: 99brfc.uwss
...
...

---------------------------------
(1) fit polynomial
(2) fit gaussian+continuum
(x) fit gaussian+continuum (cte)
(3) fit gaussian+cte
(c) fit Cauchy function+continuum
(p) fit pseudo-continuum
---------------------------------
Option [1] ? x

Enter points to fit continuum:
Select region by keyboard or mourse (k/m) [m] ? 
Enter X to exit from selection mode
Press mouse button (point #1)...  X-position:    617
Press mouse button (point #2)...  X-position:    650
Enter X to exit from selection mode
Press mouse button (point #1)...  X-position:    757
Press mouse button (point #2)...  X-position:    791
Enter X to exit from selection mode
Press mouse button (point #1)...Number of points for fit =     69

Coefficients from fit: (y=a0+a1*x+a2*x*x+...)
> a(1) :    4.29963684    

Enter points to fit gaussian:
Enter X to exit from selection mode
Press mouse button (point #1)...  X-position:    690
Press mouse button (point #2)...  X-position:    714
Enter X to exit from selection mode
Press mouse button (point #1)...Number of points for fit =     25
YRMSTOL for DOWNHILL [9.99999975E-05] ? 
Coefficients from fit: y=a*exp[-(x-x0)^2/(2 sig^2)]
> a   (+ rmsDOWNHILL):    12101.5049       5.63818612E-04
> x0  (+ rmsDOWNHILL):    701.062256       1.28760760E-07
> sig (+ rmsDOWNHILL):    2.27711821       0.00000000    
 x0 - 1 sig, x0 + 1 sig :    698.785156       703.339355    
 x0 - 2 sig, x0 + 2 sig :    696.507996       705.616516    
 x0 - 3 sig, x0 + 3 sig :    694.230896       707.893616    
 x0 - 4 sig, x0 + 4 sig :    691.953796       710.170715    
 x0 - 6 sig, x0 + 6 sig :    687.399536       714.724976    
 x0 - 8 sig, x0 + 8 sig :    682.845337       719.279175   
```
 
 
<span style="color:darkblue"> The spatial profile can be fitted with a Gaussian funtion. We took x0+/-6 sigma, which corresponds with the scans 687-715.</span>

<span style="color:darkblue"> Then 1D spectrum extraction was then carried out with the ‘adnsc’ function. </span>

```
$ R5-adnsc
*******************************************************************************
REDUCEMEv5.0                   Welcome to adnsc       Version: 28-November-1996
-------------------------------------------------------------------------------

Work with error images (y/n) [y] ? 
Output into a single spectrum (y/n) [y] ? 
Input file name? 99brfc.uwss
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: [not found]
Input error file name [99brfce.uwss] ? 
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: [not found] @ERROR@

* If you are using different factors for each spectrum, DO NOT normalize
Normalize final spectrum (y/n) [y] ? n
Scan region (0,0=EXIT) [0,0] ? 687,715
Factor [1.0] ? 
>>> Number of scans added......:     15
>>> Total number of scans added:     15
Scan region (0,0=EXIT) [0,0] ? 
Output file name? 99.sx
Output error file name [99e.sx] ?
```

### - 2.5) Atmospheric extinction correction

<span style="color:darkblue"> We changed the airmass and exposure time values of the header of the image:</span>


```

$ fitsheader 0002581299-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits -k airmass -k exptime -f
                         filename                              AIRMASS      EXPTIME
---------------------------------------------------------- ---------------- -------
0002581299-20200526-OSIRIS-OsirisLongSlitSpectroscopy.fits 1.02166566182681    80.0



$ R5-exhead
*******************************************************************************
REDUCEMEv5.0                   Welcome to exhead         Version: 24-March-1998
-------------------------------------------------------------------------------

File name to be examined? 99.sx
Image size (NSCAN,NCHAN): 1,2051
STWV    : 3644.58765
DISP    : 2.06242371
Airmass : -999.000000
Timexpos: -999.000000
Object  : [not found]
No. of characters:     11
FITSfile: [not found]
Comment : [not found]
Change head information (y/n) [n] ? y
NSCAN     (1,...,4096) [1] ? 
NCHAN     (1,...,4096) [2051] ? 
STWV     [3644.58765] ? 
DISP     [2.06242371] ? 
AIRMASS  [-999.000000] ?  1.0216657 
TIMEXPOS [-999.000000] ? 80
OBJECT  : [not found]
Change OBJECT value (y/n/e=add @ERROR@) [n] ? y
OBJECT   ? GD153
FITSFILE: [not found] 
Change FITSFILE value (y/n) [n] ? 
COMMENT : [not found] Change COMMENT value (y/n) [n] ? 
Output file name? 99h.sx

```
<span style="color:darkblue"> Finally, we corrected for atmospheric extinction with the ‘corrext’ function:</span>


```
$ R5-corrext 
*******************************************************************************
REDUCEMEv5.0                  Welcome to corrext       Version: 6-December-1996
-------------------------------------------------------------------------------

Work with error images (y/n) [n] ? y
Input file name? 99h.sx
>>> NSCAN :      1
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: GD153
Input error file name [99he.sx] ? 
>>> NSCAN :      1
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: GD153 @ERROR@
STWV = 3644.58765765                                     
DISP = 2.06242371371                                     
Are these values OK (y/n) [y] ? 
Correct from atmospheric extinction (y/n) [y] ? 
1 - Curve from file
2 - Semi-analytical function
Option (1/2) [2] ? 1
Extinction curve file name [/files/extlp.dat] ? extlp.dat
No. points read:    201
Mean air mass [1.02166569] ? 
Interpolating function...   ...OK
Correct from interstellar extinction (y/n) [y] ? n
Output file name? 99ha.sx
Output error file name [99hae.sx] ? 

```


<span style="color:darkblue">The final image was then converted again to FITS format:</span>

```
$ R5-writefits
*******************************************************************************
REDUCEMEv5.0                 Welcome to writefits          Version: 05-May-2004
-------------------------------------------------------------------------------

Input file name? 99ha.sx
>>> NSCAN :      1
>>> NCHAN :   2051
>>> STWV  : 3644.58765
>>> DISP  : 2.06242371
>>> OBJECT: GD153
Output file name? 99ha_sx.fits
Are you including the FITS header from another file (y/n) [n] ? 

```

<span style="color:darkblue">We will use a combination of Python scripts and the package **boundfit** (https://boundfit.readthedocs.io/en/latest/) (see [Cardiel 2009](https://ui.adsabs.harvard.edu/abs/2009MNRAS.396..680C/abstract)) to generate the response curve and the upper boundary fit of the stellar continuum (see `1_Response_curve_OSIRIS.ipynb`).</span>
